In [1]:
import pandas as pd
import numpy as np
import itertools
import datetime
import matplotlib.pyplot as plt
from datetime import *
import pandas_gbq
from datetime import datetime, timedelta, date
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
project_id = "perceptive-ivy-290216"

# Standard plotly imports
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
query2=f"""
SELECT
A.*,
B.RoundNumber
FROM `perceptive-ivy-290216.f1_api.results_race` A
INNER JOIN
`perceptive-ivy-290216.f1_api.schedules_update` B
ON A.GP=B.EventName AND A.Year=B.Year
# WHERE
# A.Year=2024
# AND B.Year=2024
-- AND
--  A.GP='Qatar Grand Prix'
"""

track2=pandas_gbq.read_gbq(query2,project_id,dialect='standard')

Downloading: 100%|██████████|


In [4]:
query3=f"""
SELECT
A.*,
B.RoundNumber
FROM `perceptive-ivy-290216.f1_api.results_sprint` A
INNER JOIN
`perceptive-ivy-290216.f1_api.schedules_update` B
ON A.GP=B.EventName AND A.Year=B.Year
# WHERE
# A.Year=2024
-- AND
-- A.GP='Qatar Grand Prix'
"""
track3=pandas_gbq.read_gbq(query3,project_id,dialect='standard')

Downloading: 100%|██████████|


In [107]:
year=2024

In [108]:
track_race=track2[track2["Year"]==year]
track_sprint=track3[track3["Year"]==year]

In [109]:
points_table=track_race[["RoundNumber","GP","FullName","Points"]]
points_table=points_table.fillna(0)
points_table["Points"]=points_table["Points"].astype(int)
points_table=points_table.sort_values(by=["RoundNumber","FullName"])

In [110]:
points_table.tail()

,RoundNumber,GP,FullName,Points
14470,17,Abu Dhabi Grand Prix,Pierre Gasly,4
23957,17,Abu Dhabi Grand Prix,Pietro Fittipaldi,0
18570,17,Abu Dhabi Grand Prix,Sebastian Vettel,0
15297,17,Abu Dhabi Grand Prix,Sergio Perez,0
16823,17,Abu Dhabi Grand Prix,Valtteri Bottas,18


In [111]:
points_table_sprint=track_sprint[["RoundNumber","GP","FullName","Points"]]
points_table_sprint["Points"]=points_table_sprint["Points"].astype(int)
points_table_sprint=points_table_sprint.sort_values(by=["RoundNumber","FullName"])

In [112]:
points_table=pd.concat([points_table,points_table_sprint])
# points_table[points_table["GP"]=="British Grand Prix"].sort_values("FullName").head()

In [113]:
races = points_table['GP'].drop_duplicates()
results = points_table.pivot_table(index='FullName', columns='RoundNumber', values='Points', aggfunc='sum')
results.fillna(0)

results['total_points'] = results.sum(axis=1)
results = results.sort_values(by='total_points', ascending=False)
results.drop(columns='total_points', inplace=True)

# Use race name, instead of round no., as column names
results.columns = races
results=results.fillna(0)

In [114]:
fig_pts = px.imshow(
    results,
    text_auto=True,
    aspect='auto',  # Automatically adjust the aspect ratio
    width=1200, height=700,
    # color_continuous_scale=[[0,    'rgb(198, 219, 239)'],  # Blue scale
    #                         [0.25, 'rgb(107, 174, 214)'],
    #                         [0.5,  'rgb(33,  113, 181)'],
    #                         [0.75, 'rgb(8,   81,  156)'],
    #                         [1,    'rgb(8,   48,  107)']],
    labels={'x': 'Race',
            'y': 'FullName',
            'color': 'Points'}       # Change hover texts
)
fig_pts.update_xaxes(title_text='')      # Remove axis titles
fig_pts.update_yaxes(title_text='')
fig_pts.update_yaxes(tickmode='linear')  # Show all ticks, i.e. driver names
fig_pts.update_yaxes(showgrid=True, gridwidth=1, gridcolor='LightGrey',
                 showline=False,
                 tickson='boundaries')              # Show horizontal grid only
fig_pts.update_xaxes(showgrid=False, showline=False)    # And remove vertical grid
fig_pts.update_layout(plot_bgcolor='rgba(0,0,0,0)')     # White background
fig_pts.update_layout(coloraxis_showscale=False)        # Remove legend
fig_pts.update_layout(xaxis=dict(side='top'))           # x-axis on top
fig_pts.update_layout(margin=dict(l=0, r=0, b=0, t=0))  # Remove border margins
fig_pts.update_layout(
    yaxis = dict(tickfont = dict(size=15)),
    xaxis = dict(tickfont = dict(size=13.5)),
    hoverlabel=dict(
        bgcolor="white",
        font_size=16,
        font_family="PT Sans Narrow"
    ),
    xaxis_title="<b>GP</b>",
    yaxis_title="<b>Driver</b>",
    title_x=0.5,
    font=dict(
        family="PT Sans Narrow",
        size=14,
        color="BLACK"
    ),
    title_font_family="PT Sans Narrow"
)
fig_pts.show()

In [115]:
fig_pts.write_html("Driver and GP Points Matrix for the {} F1 Season.html".format(year))

In [116]:
#Position

In [117]:
pos_table=track_race[["RoundNumber","GP","FullName","Position"]]
pos_table=pos_table.fillna(0)
pos_table["Position"]=pos_table["Position"].astype(int)
pos_table=pos_table.sort_values(by=["RoundNumber","FullName"])

In [118]:
pos_table_sprint=track_sprint[["RoundNumber","GP","FullName","Position"]]
pos_table_sprint["Position"]=pos_table_sprint["Position"].astype(int)
pos_table_sprint=pos_table_sprint.sort_values(by=["RoundNumber","FullName"])

In [119]:
races_pos = pos_table['GP'].drop_duplicates()
results_pos = pos_table.pivot_table(index='FullName', columns='RoundNumber', values='Position', aggfunc='sum')

# results['total_points'] = results.sum(axis=1)
# results = results.sort_values(by='total_points', ascending=False)
# results.drop(columns='total_points', inplace=True)

# Use race name, instead of round no., as column names
results_pos.columns = races_pos
results_pos.fillna(0)
results_pos

GP,Austrian Grand Prix,Styrian Grand Prix,Hungarian Grand Prix,British Grand Prix,70th Anniversary Grand Prix,Spanish Grand Prix,Belgian Grand Prix,Italian Grand Prix,Tuscan Grand Prix,Russian Grand Prix,Eifel Grand Prix,Portuguese Grand Prix,Emilia Romagna Grand Prix,Turkish Grand Prix,Bahrain Grand Prix,Sakhir Grand Prix,Abu Dhabi Grand Prix
FullName,,,,,,,,,,,,,,,,,
Alexander Albon,13.0,4.0,5.0,8.0,5.0,8.0,6.0,15.0,3.0,10.0,17.0,12.0,15.0,7.0,3.0,6.0,4.0
Antonio Giovinazzi,9.0,14.0,17.0,14.0,17.0,16.0,18.0,16.0,17.0,11.0,10.0,15.0,10.0,20.0,16.0,13.0,16.0
Carlos Sainz,5.0,9.0,9.0,13.0,13.0,6.0,20.0,2.0,18.0,19.0,5.0,6.0,7.0,5.0,5.0,4.0,6.0
Charles Leclerc,2.0,19.0,11.0,3.0,4.0,20.0,14.0,18.0,8.0,6.0,7.0,4.0,5.0,4.0,10.0,20.0,13.0
Daniel Ricciardo,19.0,8.0,8.0,4.0,14.0,11.0,4.0,6.0,4.0,5.0,3.0,9.0,3.0,10.0,7.0,5.0,7.0
Daniil Kvyat,12.0,10.0,12.0,18.0,10.0,12.0,11.0,9.0,7.0,8.0,15.0,19.0,4.0,12.0,11.0,7.0,11.0
Esteban Ocon,8.0,18.0,NaN,6.0,8.0,13.0,5.0,8.0,14.0,7.0,18.0,8.0,19.0,11.0,9.0,2.0,9.0
George Russell,15.0,16.0,18.0,12.0,18.0,17.0,19.0,14.0,11.0,18.0,20.0,14.0,16.0,16.0,12.0,9.0,15.0
Jack Aitken,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.0,NaN


In [120]:
fig_pos = px.imshow(
    results_pos,
    text_auto=True,
    aspect='auto',  # Automatically adjust the aspect ratio
    width=1200, height=700,
    color_continuous_scale=[[0,    'rgb(198, 219, 239)'],  # Blue scale
                            [0.25, 'rgb(107, 174, 214)'],
                            [0.5,  'rgb(33,  113, 181)'],
                            [0.75, 'rgb(8,   81,  156)'],
                            [1,    'rgb(8,   48,  107)']],
    labels={'x': 'Race',
            'y': 'Driver',
            'color': 'Position'}       # Change hover texts
)
fig_pos.update_xaxes(title_text='')      # Remove axis titles
fig_pos.update_yaxes(title_text='')
fig_pos.update_yaxes(tickmode='linear')  # Show all ticks, i.e. driver names
fig_pos.update_yaxes(showgrid=True, gridwidth=1, gridcolor='LightGrey',
                 showline=False,
                 tickson='boundaries')              # Show horizontal grid only
fig_pos.update_xaxes(showgrid=False, showline=False)    # And remove vertical grid
fig_pos.update_layout(plot_bgcolor='rgba(0,0,0,0)')     # White background
fig_pos.update_layout(coloraxis_showscale=False)        # Remove legend
fig_pos.update_layout(xaxis=dict(side='top'))           # x-axis on top
fig_pos.update_layout(margin=dict(l=0, r=0, b=0, t=0))  # Remove border margins
fig_pos.update_layout(
    yaxis = dict(tickfont = dict(size=15)),
    xaxis = dict(tickfont = dict(size=13.5)),
    xaxis_title="<b>GP</b>",
    yaxis_title="<b>Driver</b>",
    title_x=0.5,
    hoverlabel=dict(
        bgcolor="white",
        font_size=16,
        font_family="PT Sans Narrow"
    ),
    font=dict(
        family="PT Sans Narrow",
        size=14,
        color="BLACK"
    ),
    title_font_family="PT Sans Narrow"
)
fig_pos.show()

In [121]:
fig_pos.write_html("Driver and Final GP Position Matrix for the {} F1 Season.html".format(year))

In [122]:
races_pos_sprint = pos_table_sprint['GP'].drop_duplicates()
results_pos_sprint = pos_table_sprint.pivot_table(index='FullName', columns='RoundNumber', values='Position', aggfunc='sum')

# results['total_points'] = results.sum(axis=1)
# results = results.sort_values(by='total_points', ascending=False)
# results.drop(columns='total_points', inplace=True)

# Use race name, instead of round no., as column names
results_pos_sprint.columns = races_pos_sprint
results_pos_sprint.fillna(0)
results_pos_sprint

GP
FullName


In [123]:
fig_pos_sprint = px.imshow(
    results_pos_sprint,
    text_auto=True,
    width=1200, height=700,
    aspect='auto',  # Automatically adjust the aspect ratio
    color_continuous_scale=[[0,    'rgb(198, 219, 239)'],  # Blue scale
                            [0.25, 'rgb(107, 174, 214)'],
                            [0.5,  'rgb(33,  113, 181)'],
                            [0.75, 'rgb(8,   81,  156)'],
                            [1,    'rgb(8,   48,  107)']],
    labels={'x': 'Race',
            'y': 'Driver',
            'color': 'Position'}       # Change hover texts
)
fig_pos_sprint.update_xaxes(title_text='')      # Remove axis titles
fig_pos_sprint.update_yaxes(title_text='')
fig_pos_sprint.update_yaxes(tickmode='linear')  # Show all ticks, i.e. driver names
fig_pos_sprint.update_yaxes(showgrid=True, gridwidth=1, gridcolor='LightGrey',
                 showline=False,
                 tickson='boundaries')              # Show horizontal grid only
fig_pos_sprint.update_xaxes(showgrid=False, showline=False)    # And remove vertical grid
fig_pos_sprint.update_layout(plot_bgcolor='rgba(0,0,0,0)')     # White background
fig_pos_sprint.update_layout(coloraxis_showscale=False)        # Remove legend
fig_pos_sprint.update_layout(xaxis=dict(side='top'))           # x-axis on top
fig_pos_sprint.update_layout(margin=dict(l=0, r=0, b=0, t=0))  # Remove border margins
fig_pos_sprint.update_layout(
    yaxis = dict(tickfont = dict(size=15)),
    xaxis = dict(tickfont = dict(size=13.5)),
    xaxis_title="<b>GP</b>",
    yaxis_title="<b>Driver</b>",
    title_x=0.5,
    hoverlabel=dict(
        bgcolor="white",
        font_size=16,
        font_family="PT Sans Narrow"
    ),
    font=dict(
        family="PT Sans Narrow",
        size=14,
        color="BLACK"
    ),
    title_font_family="PT Sans Narrow"
)
fig_pos_sprint.show()

In [124]:
fig_pos_sprint.write_html("Driver and Final Sprint Position Matrix for the {} F1 Season.html".format(year))

In [125]:
# Grid Position

In [126]:
grid_pos_table=track_race[["RoundNumber","GP","FullName","Position"]]
grid_pos_table=grid_pos_table.fillna(0)
grid_pos_table["Position"]=grid_pos_table["Position"].astype(int)
grid_pos_table=grid_pos_table.sort_values(by=["RoundNumber","FullName"])

In [127]:
grid_pos_table_sprint=track_sprint[["RoundNumber","GP","FullName","Position"]]
grid_pos_table_sprint["Position"]=grid_pos_table_sprint["Position"].astype(int)
grid_pos_table_sprint=grid_pos_table_sprint.sort_values(by=["RoundNumber","FullName"])

In [128]:
races_grid_pos = grid_pos_table['GP'].drop_duplicates()
results_grid_pos = grid_pos_table.pivot_table(index='FullName', columns='RoundNumber', values='Position', aggfunc='sum')

# results['total_points'] = results.sum(axis=1)
# results = results.sort_values(by='total_points', ascending=False)
# results.drop(columns='total_points', inplace=True)

# Use race name, instead of round no., as column names
results_grid_pos.columns = races_grid_pos
results_grid_pos.fillna(0)
results_grid_pos

GP,Austrian Grand Prix,Styrian Grand Prix,Hungarian Grand Prix,British Grand Prix,70th Anniversary Grand Prix,Spanish Grand Prix,Belgian Grand Prix,Italian Grand Prix,Tuscan Grand Prix,Russian Grand Prix,Eifel Grand Prix,Portuguese Grand Prix,Emilia Romagna Grand Prix,Turkish Grand Prix,Bahrain Grand Prix,Sakhir Grand Prix,Abu Dhabi Grand Prix
FullName,,,,,,,,,,,,,,,,,
Alexander Albon,13.0,4.0,5.0,8.0,5.0,8.0,6.0,15.0,3.0,10.0,17.0,12.0,15.0,7.0,3.0,6.0,4.0
Antonio Giovinazzi,9.0,14.0,17.0,14.0,17.0,16.0,18.0,16.0,17.0,11.0,10.0,15.0,10.0,20.0,16.0,13.0,16.0
Carlos Sainz,5.0,9.0,9.0,13.0,13.0,6.0,20.0,2.0,18.0,19.0,5.0,6.0,7.0,5.0,5.0,4.0,6.0
Charles Leclerc,2.0,19.0,11.0,3.0,4.0,20.0,14.0,18.0,8.0,6.0,7.0,4.0,5.0,4.0,10.0,20.0,13.0
Daniel Ricciardo,19.0,8.0,8.0,4.0,14.0,11.0,4.0,6.0,4.0,5.0,3.0,9.0,3.0,10.0,7.0,5.0,7.0
Daniil Kvyat,12.0,10.0,12.0,18.0,10.0,12.0,11.0,9.0,7.0,8.0,15.0,19.0,4.0,12.0,11.0,7.0,11.0
Esteban Ocon,8.0,18.0,NaN,6.0,8.0,13.0,5.0,8.0,14.0,7.0,18.0,8.0,19.0,11.0,9.0,2.0,9.0
George Russell,15.0,16.0,18.0,12.0,18.0,17.0,19.0,14.0,11.0,18.0,20.0,14.0,16.0,16.0,12.0,9.0,15.0
Jack Aitken,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.0,NaN


In [129]:
fig_grid_pos = px.imshow(
    results_grid_pos,
    text_auto=True,
    aspect='auto',  # Automatically adjust the aspect ratio
    width=1200, height=700,
    color_continuous_scale=[[0,    'rgb(198, 219, 239)'],  # Blue scale
                            [0.25, 'rgb(107, 174, 214)'],
                            [0.5,  'rgb(33,  113, 181)'],
                            [0.75, 'rgb(8,   81,  156)'],
                            [1,    'rgb(8,   48,  107)']],
    labels={'x': 'Race',
            'y': 'Driver',
            'color': 'Position'}       # Change hover texts
)
fig_grid_pos.update_xaxes(title_text='')      # Remove axis titles
fig_grid_pos.update_yaxes(title_text='')
fig_grid_pos.update_yaxes(tickmode='linear')  # Show all ticks, i.e. driver names
fig_grid_pos.update_yaxes(showgrid=True, gridwidth=1, gridcolor='LightGrey',
                 showline=False,
                 tickson='boundaries')              # Show horizontal grid only
fig_grid_pos.update_xaxes(showgrid=False, showline=False)    # And remove vertical grid
fig_grid_pos.update_layout(plot_bgcolor='rgba(0,0,0,0)')     # White background
fig_grid_pos.update_layout(coloraxis_showscale=False)        # Remove legend
fig_grid_pos.update_layout(xaxis=dict(side='top'))           # x-axis on top
fig_grid_pos.update_layout(margin=dict(l=0, r=0, b=0, t=0))  # Remove border margins
fig_grid_pos.update_layout(
    yaxis = dict(tickfont = dict(size=15)),
    xaxis = dict(tickfont = dict(size=13.5)),
    xaxis_title="<b>GP</b>",
    yaxis_title="<b>Driver</b>",
    title_x=0.5,
    hoverlabel=dict(
        bgcolor="white",
        font_size=16,
        font_family="PT Sans Narrow"
    ),
    font=dict(
        family="PT Sans Narrow",
        size=14,
        color="BLACK"
    ),
    title_font_family="PT Sans Narrow"
)
fig_grid_pos.show()

In [130]:
fig_grid_pos.write_html("Driver and Race Grid Position Matrix for the {} F1 Season.html".format(year))

In [131]:
races_grid_pos_sprint = grid_pos_table_sprint['GP'].drop_duplicates()
results_grid_pos_sprint = grid_pos_table_sprint.pivot_table(index='FullName', columns='RoundNumber', values='Position', aggfunc='sum')

# results['total_points'] = results.sum(axis=1)
# results = results.sort_values(by='total_points', ascending=False)
# results.drop(columns='total_points', inplace=True)

# Use race name, instead of round no., as column names
results_grid_pos_sprint.columns = races_grid_pos_sprint
results_grid_pos_sprint.fillna(0)
results_grid_pos_sprint

GP
FullName


In [132]:
fig_grid_sprint_pos = px.imshow(
    results_grid_pos_sprint,
    text_auto=True,
    aspect='auto',  # Automatically adjust the aspect ratio
    width=1200, height=700,
    color_continuous_scale=[[0,    'rgb(198, 219, 239)'],  # Blue scale
                            [0.25, 'rgb(107, 174, 214)'],
                            [0.5,  'rgb(33,  113, 181)'],
                            [0.75, 'rgb(8,   81,  156)'],
                            [1,    'rgb(8,   48,  107)']],
    labels={'x': 'Race',
            'y': 'Driver',
            'color': 'Position'}       # Change hover texts
)
fig_grid_sprint_pos.update_xaxes(title_text='')      # Remove axis titles
fig_grid_sprint_pos.update_yaxes(title_text='')
fig_grid_sprint_pos.update_yaxes(tickmode='linear')  # Show all ticks, i.e. driver names
fig_grid_sprint_pos.update_yaxes(showgrid=True, gridwidth=1, gridcolor='LightGrey',
                 showline=False,
                 tickson='boundaries')              # Show horizontal grid only
fig_grid_sprint_pos.update_xaxes(showgrid=False, showline=False)    # And remove vertical grid
fig_grid_sprint_pos.update_layout(plot_bgcolor='rgba(0,0,0,0)')     # White background
fig_grid_sprint_pos.update_layout(coloraxis_showscale=False)        # Remove legend
fig_grid_sprint_pos.update_layout(xaxis=dict(side='top'))           # x-axis on top
fig_grid_sprint_pos.update_layout(margin=dict(l=0, r=0, b=0, t=0))  # Remove border margins
fig_grid_sprint_pos.update_layout(
    yaxis = dict(tickfont = dict(size=15)),
    xaxis = dict(tickfont = dict(size=13.5)),
    xaxis_title="<b>GP</b>",
    yaxis_title="<b>Driver</b>",
    title_x=0.5,
    hoverlabel=dict(
        bgcolor="white",
        font_size=16,
        font_family="PT Sans Narrow"
    ),
    font=dict(
        family="PT Sans Narrow",
        size=14,
        color="BLACK"
    ),
    title_font_family="PT Sans Narrow"
)
fig_grid_sprint_pos.show()

In [133]:
fig_grid_sprint_pos.write_html("Driver and Sprint Race Grid Position Matrix for the {} F1 Season.html".format(year))